In [19]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [179]:

cnx = mysql.connector.connect(user='root', password='yourpassb', host='localhost', database='mysql')
query = "SELECT * FROM economic_data LIMIT 10;"
df = pd.read_sql(query, cnx)
describe=df.describe()
print(df)



   CountryName CountryCode                                         SeriesName  \
0  Afghanistan         AFG               Death rate, crude (per 1,000 people)   
1  Afghanistan         AFG  Debt service on external debt, public and publ...   
2  Afghanistan         AFG           Exports of goods and services (% of GDP)   
3  Afghanistan         AFG   Exports of goods and services (BoP, current US$)   
4  Afghanistan         AFG  Foreign direct investment, net outflows (% of ...   
5  Afghanistan         AFG  Foreign direct investment, net inflows (% of GDP)   
6  Afghanistan         AFG  Foreign direct investment, net (BoP, current US$)   
7  Afghanistan         AFG  Foreign direct investment, net outflows (BoP, ...   
8  Afghanistan         AFG                   GDP per capita growth (annual %)   
9  Afghanistan         AFG                       GDP per capita (current US$)   

             SeriesCode        YR2013        YR2014        YR2015  \
0        SP.DYN.CDRT.IN         7.478  

C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\3939403189.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


by calculating the possible correlation between death rate, GDP and Unemployment rate it turn out that there is not strong correlation between them

In [141]:
def fetch_indicator_avg(indicator_name):
    query = f"""
    SELECT CountryName, YR2013, YR2014, YR2015, YR2016, YR2017, YR2018, YR2019, YR2020, YR2021, YR2022
    FROM economic_data
    WHERE SeriesName = '{indicator_name}';
    """
    df = pd.read_sql(query, cnx)
    year_columns = ['YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018', 'YR2019', 'YR2020', 'YR2021', 'YR2022']
    for col in year_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df[f'Mean{indicator_name.replace(" ", "")}'] = df[year_columns].mean(axis=1)
    return df[['CountryName', f'Mean{indicator_name.replace(" ", "")}']]

# Fetch and calculate average for each indicator
death_rate_df = fetch_indicator_avg('Death rate, crude (per 1,000 people)')
gdp_per_capita_df = fetch_indicator_avg('GDP per capita (current US$)')
unemployment_rate_df = fetch_indicator_avg('Unemployment, total (% of total labor force) (national estimate)')

# Merge the dataframes
combined_df = death_rate_df.merge(gdp_per_capita_df, on='CountryName').merge(unemployment_rate_df, on='CountryName')


# Check for correlation
correlation_matrix = combined_df.corr()
unemployment_rate_df

                                                    MeanDeathrate,crude(per1,000people)  \
MeanDeathrate,crude(per1,000people)                                            1.000000   
MeanGDPpercapita(currentUS$)                                                  -0.079663   
MeanUnemployment,total(%oftotallaborforce)(nati...                             0.149047   

                                                    MeanGDPpercapita(currentUS$)  \
MeanDeathrate,crude(per1,000people)                                    -0.079663   
MeanGDPpercapita(currentUS$)                                            1.000000   
MeanUnemployment,total(%oftotallaborforce)(nati...                     -0.138636   

                                                    MeanUnemployment,total(%oftotallaborforce)(nationalestimate)  
MeanDeathrate,crude(per1,000people)                                                          0.149047             
MeanGDPpercapita(currentUS$)                                        

C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\2894908654.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\2894908654.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\2894908654.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\2894908654.py:38: FutureWarning: The default value

Below cell helps us to find country with highest unemployment rate

In [129]:
query = """
SELECT CountryName, YR2013, YR2014, YR2015, YR2016, YR2017, YR2018, YR2019, YR2020, YR2021, YR2022
FROM economic_data
WHERE SeriesName = 'Unemployment, total (% of total labor force) (national estimate)';  -- Replace with the actual indicator name for unemployment rate in your dataset
"""
df_unemploy = pd.read_sql(query, cnx)


year_columns = ['YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018', 'YR2019', 'YR2020', 'YR2021', 'YR2022']
for col in year_columns:
    df_unemploy[col] = pd.to_numeric(df_unemploy[col], errors='coerce')

df_unemploy['MaxUnemploymentRate'] = df_unemploy[year_columns].max(axis=1)

# Find the country with the highest maximum unemployment rate
country_with_max_unemployment = df_unemploy[df_unemploy['MaxUnemploymentRate'] == df_unemploy['MaxUnemploymentRate'].max()]

# Print the result
print(country_with_max_unemployment[['CountryName', 'MaxUnemploymentRate']])

    CountryName  MaxUnemploymentRate
105      Kosovo               35.264


C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\57231244.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_unemploy = pd.read_sql(query, cnx)


In [180]:
def fetch_avg_indicator(indicator_name):
    query = f"""
    SELECT CountryName, YR2013, YR2014, YR2015, YR2016, YR2017, YR2018, YR2019, YR2020, YR2021, YR2022
    FROM economic_data
    WHERE SeriesName = '{indicator_name}';
    """
    df = pd.read_sql(query, cnx)
    year_columns = ['YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018', 'YR2019', 'YR2020', 'YR2021', 'YR2022']
    for col in year_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df[f'Avg{indicator_name.replace(" ", "")}'] = df[year_columns].mean(axis=1)
    return df[['CountryName', f'Avg{indicator_name.replace(" ", "")}']]

# Fetch and calculate average for each indicator
avg_inflation_df = fetch_avg_indicator('Inflation, consumer prices (annual %)')
avg_poverty_headcount_df = fetch_avg_indicator('Poverty headcount ratio at national poverty lines (% of population)')

# Merge the dataframes
combined_df = avg_inflation_df.merge(avg_poverty_headcount_df, on='CountryName')

# Close the database connection
cnx.close()

# Check for correlation
correlation_matrix = combined_df.corr()
print(correlation_matrix)

                                                    AvgInflation,consumerprices(annual%)  \
AvgInflation,consumerprices(annual%)                                            1.000000   
AvgPovertyheadcountratioatnationalpovertylines(...                              0.222261   

                                                    AvgPovertyheadcountratioatnationalpovertylines(%ofpopulation)  
AvgInflation,consumerprices(annual%)                                                         0.222261              
AvgPovertyheadcountratioatnationalpovertylines(...                                           1.000000              


C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\2464665811.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\2464665811.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\2464665811.py:25: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = combined_df.corr()


The correlation is positive but relatively weak. This suggests that while there might be some degree of association between higher inflation and higher poverty rates, other factors are also likely at play influencing poverty levels

In [166]:
query = """
SELECT CountryName, YR2013, YR2014, YR2015, YR2016, YR2017, YR2018, YR2019, YR2020, YR2021, YR2022
FROM economic_data
WHERE SeriesName = 'Exports of goods and services (BoP, current US$)';
"""
df = pd.read_sql(query, cnx)



# Convert the year columns to numeric and handle non-numeric values
year_columns = ['YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018', 'YR2019', 'YR2020', 'YR2021', 'YR2022']
for col in year_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate the average export value for each country
df['AvgExports'] = df[year_columns].mean(axis=1)

# Find the country with the highest average export value
country_with_min_avg_exports = df[df['AvgExports'] == df['AvgExports'].min()]

# Print the result
print(country_with_min_avg_exports[['CountryName', 'AvgExports']])

    CountryName    AvgExports
201      Tuvalu  6.133880e+06


C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\1452330957.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


Now that Tuvala has the least exports among all countries. Lets determine whether the country economy is mainly rely on domestic market or export

In [177]:

# Function to fetch average indicator value for Tuvalu
def fetch_avg_indicator_for_tuvalu(indicator_name):
    query = f"""
    SELECT YR2013, YR2014, YR2015, YR2016, YR2017, YR2018, YR2019, YR2020, YR2021, YR2022
    FROM economic_data
    WHERE SeriesName = '{indicator_name}' AND CountryName = 'Tuvalu';
    """
    df = pd.read_sql(query, cnx)
    if df.empty:
        return None  # Returns None if there's no data for Tuvalu
    year_columns = ['YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018', 'YR2019', 'YR2020', 'YR2021', 'YR2022']
    for col in year_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df[year_columns].mean(axis=1).iloc[0]

# Fetch average export and GDP for Tuvalu
avg_exports = fetch_avg_indicator_for_tuvalu('Exports of goods and services (BoP, current US$)')
avg_gdp = fetch_avg_indicator_for_tuvalu('GDP per capita (current US$)')

# Close the database connection
cnx.close()

# Print the results
print(f"Average Exports for Tuvalu (2013-2022): {avg_exports/11204}")
print(f"Average GDP for Tuvalu (2013-2022): {avg_gdp}")

# Comparing the two values
if avg_exports is not None and avg_gdp is not None:
    comparison = 547.32 / avg_gdp
    print(f"Ratio of Average Exports to GDP for Tuvalu: {comparison}")
else:
    print("Insufficient data for comparison.")


Average Exports for Tuvalu (2013-2022): 547.4723136558372
Average GDP for Tuvalu (2013-2022): 4312.3689445
Ratio of Average Exports to GDP for Tuvalu: 0.12691863962568709


C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\2745276048.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11884\2745276048.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


comparison of ration of average exports to acerage GDP indicates that the export sector in Tuvalu, on a per capita basis, is relatively small compared to the overall size of its economy per capita. It implies that exports are a contributing but not a dominant part of Tuvalu's economy